# Train, Validate --> Train, Test

In this exercise, you will perform empirical comparison of the results of a ten-fold cross validated model with a fully trained model.

## Notes and Guidelines
* Read a dataset from disk and use it for a classification task.
* Construct a Gaussian Naive Bayes classifier and fit it to the phoneme dataset provided.
* Save and re-load a trained classifier.
* Compare K-fold cross-validation scores with the success rate of a fully-trained model.


### Dataset
* Dataset acquired from [KEEL](http://sci2s.ugr.es/keel/dataset.php?cod=105), an excellent resource for finding 'toy' datasets (and a few more serious ones).
    * A description of the dataset is provided at the above link - **read it.**
    * Excerpt: 
    *The aim of this dataset is to distinguish between nasal (class 0) and oral sounds (class 1).
    The class distribution is 3,818 samples in class 0 and 1,586 samples in class 1.
    The phonemes are transcribed as follows: sh as in she, dcl as in dark, iy as the vowel in she, aa as the vowel in dark, and ao as the first vowel in water.*
    
* It is not necessary to fully understand the nature or context of the values in the dataset - only that there are five columns of input (featural) data and one column of output (class) data.

## Handling imports and dataset inclusion

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib as jb

# locate dataset
DATASET = '/dsa/data/all_datasets/phoneme.csv'  # phoneme classification dataset
assert os.path.exists(DATASET)  # check if the file actually exists

## Constructing DataFrame from raw dataset

<span style="background:yellow">**Note**</span>: Variable `dataset` should be used for the dataframe.

In [2]:
# read_table() is like read_csv(), but generalized for delimited file formats
# and calling .sample() with a fraction of 1 shuffles the data
dataset = pd.read_table(DATASET, sep=',', engine='c', header=0).sample(frac=1)

# verify dataset shape
print("Dataset shape: ", dataset.shape)

Dataset shape:  (5404, 6)


In [3]:
dataset.head()

,Aa,Ao,Dcl,Iy,Sh,Class
4966,0.477,0.953,1.616,-0.338,-0.166,0
3121,3.580,0.496,0.327,0.344,-0.119,0
4983,0.214,0.744,0.912,2.221,-1.086,1
627,1.509,1.501,0.560,1.513,-0.407,0
2054,0.516,1.952,1.204,-0.345,-0.176,0


## Splitting data into training and test sets

In [4]:
"""
The below is only necessary if you are interested in visualizing
the data or providing neatly-labeled output within the program.

# extract labels from column headers
phonemes = dataset.columns[0:5].tolist()  # Feature labels
labels = {0: 'Nasal', 1: 'Oral'}  # Class labels
"""

# extract target data from primary data frame
X = dataset.iloc[:, 0:5]  # iloc is used here for numeric indexing
y = dataset.loc[:, "Class"]  # loc can be used for label-based indexing

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training shapes (X, y): ", X_train.shape, y_train.shape)
print("Testing shapes (X, y): ", X_test.shape, y_test.shape)

Training shapes (X, y):  (4323, 5) (4323,)
Testing shapes (X, y):  (1081, 5) (1081,)


## Constructing the classifier and running automated cross-validation

In [6]:
# Your code below this line
# --------------------------
classifier = GaussianNB()
scores = cross_val_score(classifier,X_train,y_train,cv=10)

print(scores)





[ 0.74595843  0.72055427  0.7482679   0.75462963  0.77777778  0.78935185
  0.7662037   0.73842593  0.75231481  0.78009259]


## Training the classifier and pickling to disk

In [8]:
# Your code below this line
# --------------------------
fitted = classifier.fit(X_train,y_train)
from sklearn.externals import joblib
joblib.dump(fitted,"Gaussian.Phoneme.plk")



['Gaussian.Phoneme.plk']

## Unpickling the model and making predictions

#### Fill in the `# ???` with proper code

In [11]:
# Your code below this line
# --------------------------

# load pickled model
loaded_model = joblib.load("Gaussian.Phoneme.plk")

# make predictions with freshly loaded model
predictions = loaded_model.predict(X_test)

# verify input and output shape are appropriate
print("Input vs. output shape:")
print(X_test.shape, predictions.shape)




Input vs. output shape:
(1081, 5) (1081,)


## Performing final performance comparison

In [12]:
# tally up right + wrong 'guesses' by model
true, false = 0, 0
for i, j in zip(y_test, predictions):
    # print(i, j)
    if i == j:
        true += 1
    else:
        false += 1

# report results numerically and by percentage
true_percent = true / (true + false) * 100
print("Correct guesses: " + str(true) + "\nIncorrect guesses: " + str(false))
print("Percent correct: " + str(true_percent))

# compare to average of cross-validation scores
avg_cv = np.sum(scores) / len(scores) * 100
print("Percent cross-validation score (10 folds, average): " + str(avg_cv))

Correct guesses: 788
Incorrect guesses: 293
Percent correct: 72.895467160037
Percent cross-validation score (10 folds, average): 75.7357689676


## Measure performance using Scikit Learn modules 
#### (see Module 1 labs)
Compute and display the following:
 1. Compute Confusion Matrix
 1. Accuracy
 1. Precision
 1. $F_1$-Score

In [19]:
# Your code below this line
# --------------------------
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test,predictions,average='micro')





(0.72895467160036997, 0.72895467160036997, 0.72895467160036997, None)

In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,np.round(predictions).astype('i4'))

array([[562, 196],
       [ 97, 226]])

In [20]:
loaded_model.score(X_test,y_test)# Accuracy

0.72895467160036997

## Conclusions ?

How did your trained model perform relative to your expectations based on the cross-validation?
Provide your answer in the cell below.

# Save your notebook!